In [1]:
import pandas as pd
import numpy as np
import os
import re

In [2]:
currdir = os.getcwd()
regex_name = r'(([A-Z])\w+)'
batchrun = '600s-run1'

In [3]:
SURVEY_ZONE_MAPPING = {'30': (2, 2, 0), '22': (3, 1, 2), '2c': (2, 0, 0), '28': (2, 2, 1), '25': (3, 2, 1), '26': (3, 1, 1), '27': (3, 0, 1), '20': (2, 2, 2), '21': (3, 2, 2), '1e': (3, 1, 2), '23': (3, 0, 2), '24': (2, 0, 1), '29': (3, 2, 1), '1a': (3, 1, 3), '0': (1, 0, 3), '3': (3, 1, 0), '2': (3, 2, 0), '5': (1, 0, 0), '1d': (3, 2, 2), '7': (2, 1, 1), '1f': (3, 0, 2), '9': (3, 1, 1), '8': (3, 2, 1), '3a': (3, 1, 3), '1c': (2, 0, 2), '4': (3, 0, 0), 'a': (3, 0, 1), '6': (1, 0, 1), '39': (3, 2, 3), '12': (1, 0, 2), '3b': (3, 0, 3), '1b': (3, 0, 3), 'b': (1, 0, 2), '13': (1, 0, 2), 'd': (3, 2, 2), '11': (1, 0, 3), '10': (1, 0, 3), 'c': (2, 1, 2), '38': (2, 0, 3), '15': (1, 0, 1), '14': (1, 0, 1), '17': (1, 0, 0), 'f': (3, 0, 2), '19': (3, 2, 3), '32': (3, 1, 0), '31': (3, 2, 0), '16': (1, 0, 0), '37': (3, 0, 3), '36': (3, 1, 3), '35': (3, 2, 3), '34': (2, 1, 3), '2d': (3, 2, 0), '2e': (3, 1, 0), '2f': (3, 0, 0), '1': (2, 1, 0), '2a': (3, 1, 1), '2b': (3, 0, 1), '18': (2, 2, 3), '33': (3, 0, 0), 'e': (3, 1, 2)}

## Checking out the Excel sheet (Obtaining list of columns)

In [4]:
filename = currdir + '/experiments/sim-results/result_d450_a0.5_b0.5_c0.xls'
df_tmp = pd.read_excel(filename, sheet_name='Speed').fillna(0)

In [5]:
df_tmp.head()

,t,0x20 - approach2_lane3,0x7 - approach1_lane2,0x5 - exit0_lane2,0x25 - approach1_lane1,0x19 - approach3_lane3,0x32 - approach0_lane3,0x2 - approach0_lane2,0x28 - approach1_lane3,0x17 - exit0_lane1,...,0x39 - approach3_lane1,0x27 - approach1_lane1,0x15 - exit1_lane1,0x10 - exit3_lane3,0x12 - exit2_lane3,0x13 - exit2_lane1,0x14 - exit1_lane3,0x2e - approach0_lane1,0xc - approach2_lane2,0x18 - approach3_lane3
0,60,6.043523,10.383673,68.210526,28.514118,37.319226,49.118400,50.865231,7.571339,69.120000,...,27.221082,69.893333,69.120000,58.320000,45.360000,41.040000,50.885053,36.566710,10.528290,22.159543
1,120,7.358991,9.770697,68.086957,10.923718,41.024681,53.402323,45.694214,8.097347,64.800000,...,13.348800,52.663814,63.803077,50.544000,45.360000,46.902857,53.712000,32.329925,9.943385,21.248877
2,180,8.374154,8.940575,64.536585,4.858442,36.272317,53.484480,39.984762,7.781410,59.245714,...,5.096458,44.402824,58.060800,52.488000,44.064000,47.880000,53.712000,16.896000,9.257634,15.250247
3,240,8.876712,9.014173,64.800000,3.212478,35.839405,35.200780,39.211472,7.957738,60.025263,...,3.244240,33.481161,56.043243,46.042105,45.360000,48.282353,51.474000,10.076160,9.170757,11.751273
4,300,9.441060,8.756522,63.200000,2.540449,33.109818,36.970839,35.801977,8.004662,58.060800,...,2.575650,14.651553,55.013878,45.297087,43.985455,50.811429,51.376696,7.209374,8.961580,11.357925


In [6]:
# Filter out some of the rows

delay_rows = ["{} Delay".format(d) for d in range(60,601,60)]
delay_rename = dict([
    ("{} Delay".format(d), d)
    for d in range(60,601,60)
])

thru_rows = ["{} Throughput".format(d) for d in range(60,601,60)]
thru_rename = dict([
    ("{} Throughput".format(d), d)
    for d in range(60,601,60)
])

vol_rows = [d for d in range(60,601,60)]

In [7]:
# Filter out some of the columns

survey_zones = df_tmp.columns.values[1:]

entrances = [x for x in survey_zones if 'approach' in x]
exits = [y for y in survey_zones if 'exit' in y]

print("{} approach zones, {} exit zones".format(len(entrances), len(exits)))

48 approach zones, 12 exit zones


In [8]:
reg_approach = re.compile('0x(\w+) - approach.*')
reg_exit = re.compile('0x(\w+) - exit.*')

entrances_rename = dict([
    (k, SURVEY_ZONE_MAPPING[reg_approach.match(k).group(1)])
    for k in entrances
])

exits_rename = dict([
    (k, SURVEY_ZONE_MAPPING[reg_exit.match(k).group(1)])
    for k in exits
])

## Checking out the Volume sheet

In [9]:
df_tmp = pd.read_excel(filename, sheet_name='Volume').fillna(0)

In [10]:
df_tmp

,t,0x16 - exit0_lane3,0x10 - exit3_lane3,0x1 - approach0_lane2,0x2 - approach0_lane2,0x3 - approach0_lane2,0x4 - approach0_lane2,0x28 - approach1_lane3,0x29 - approach1_lane3,0x2a - approach1_lane3,...,0x37 - approach3_lane2,0x15 - exit1_lane1,0xc - approach2_lane2,0xd - approach2_lane2,0xe - approach2_lane2,0xf - approach2_lane2,0x24 - approach1_lane1,0x25 - approach1_lane1,0x26 - approach1_lane1,0x27 - approach1_lane1
0,60,8.000000,9.00,7.000,8.000000,9.000000,9.000000,8.00,8.00,8.00,...,12.000000,2.000000,14.00,15.000000,16.000000,16.000000,4.000000,5.000000,5.000000,5.000000
1,120,6.000000,4.00,19.000,19.000000,19.000000,19.000000,6.00,6.00,6.00,...,13.000000,6.000000,17.00,18.000000,17.000000,17.000000,4.000000,7.000000,7.000000,7.000000
2,180,7.000000,5.00,14.000,13.000000,13.000000,14.000000,10.00,10.00,10.00,...,10.000000,6.000000,18.00,18.000000,18.000000,19.000000,3.000000,3.000000,4.000000,4.000000
3,240,7.000000,12.00,15.000,15.000000,14.000000,13.000000,5.00,5.00,5.00,...,23.000000,6.000000,15.00,14.000000,14.000000,13.000000,3.000000,3.000000,6.000000,7.000000
4,300,9.000000,10.00,19.000,19.000000,19.000000,20.000000,10.00,10.00,10.00,...,13.000000,6.000000,18.00,17.000000,19.000000,19.000000,3.000000,3.000000,3.000000,3.000000
5,360,6.000000,9.00,14.000,14.000000,15.000000,15.000000,4.00,4.00,4.00,...,24.000000,6.000000,15.00,15.000000,13.000000,13.000000,3.000000,3.000000,3.000000,6.000000
6,420,11.000000,13.00,15.000,17.000000,16.000000,15.000000,13.00,14.00,14.00,...,14.000000,7.000000,9.00,13.000000,14.000000,15.000000,6.000000,5.000000,3.000000,2.000000
7,480,6.000000,9.00,15.000,15.000000,16.000000,17.000000,4.00,3.00,3.00,...,20.000000,11.000000,16.00,16.000000,16.000000,15.000000,8.000000,8.000000,8.000000,8.000000
8,540,8.000000,9.00,15.000,18.000000,18.000000,18.000000,8.00,8.00,8.00,...,13.000000,10.000000,13.00,11.000000,11.000000,13.000000,9.000000,8.000000,8.000000,7.000000
9,600,9.000000,4.00,14.000,17.000000,21.000000,23.000000,10.00,10.00,10.00,...,16.000000,10.000000,15.00,14.000000,13.000000,11.000000,7.000000,8.000000,8.000000,9.000000


In [11]:
vol_rows = [d for d in range(60,601,60)]

df_tmp1 = df_tmp[df_tmp.t.isin(vol_rows)]
df_tmp1 = df_tmp1.sort_values(by='t').set_index('t')[entrances].stack().reset_index()
df_tmp1['level_1'] = df_tmp1['level_1'].map(entrances_rename)
df_tmp1 = df_tmp1.groupby(['t', 'level_1']).sum().reset_index()

df_volume = df_tmp1.pivot(index='t', columns='level_1', values=0)
df_volume

## ADD THIS LATER WHEN RESULTS ARE OUT

level_1,"(2, 0, 0)","(2, 0, 1)","(2, 0, 2)","(2, 0, 3)","(2, 1, 0)","(2, 1, 1)","(2, 1, 2)","(2, 1, 3)","(2, 2, 0)","(2, 2, 1)",...,"(3, 0, 2)","(3, 0, 3)","(3, 1, 0)","(3, 1, 1)","(3, 1, 2)","(3, 1, 3)","(3, 2, 0)","(3, 2, 1)","(3, 2, 2)","(3, 2, 3)"
t,,,,,,,,,,,,,,,,,,,,,
60,6.0,4.0,3.0,6.0,7.0,16.0,14.0,11.0,9.0,8.0,...,30.0,22.0,27.0,29.0,30.0,22.0,26.0,29.0,29.0,20.0
120,6.0,4.0,6.0,3.0,19.0,12.0,17.0,13.0,4.0,6.0,...,32.0,28.0,32.0,26.0,31.0,28.0,30.0,25.0,32.0,28.0
180,6.0,3.0,5.0,3.0,14.0,11.0,18.0,11.0,7.0,10.0,...,27.0,25.0,29.0,24.0,26.0,24.0,27.0,24.0,25.0,23.0
240,6.0,3.0,5.0,3.0,15.0,13.0,15.0,21.0,13.0,5.0,...,30.0,35.0,33.0,24.0,30.0,32.0,34.0,21.0,26.0,31.0
300,6.0,3.0,5.0,3.0,19.0,20.0,18.0,14.0,8.0,10.0,...,36.0,26.0,31.0,34.0,34.0,26.0,33.0,34.0,33.0,26.0
360,6.0,3.0,5.0,3.0,14.0,11.0,15.0,22.0,10.0,4.0,...,22.0,32.0,33.0,18.0,22.0,30.0,30.0,18.0,24.0,30.0
420,7.0,6.0,7.0,6.0,15.0,14.0,9.0,13.0,11.0,13.0,...,32.0,24.0,33.0,33.0,31.0,25.0,34.0,35.0,29.0,26.0
480,10.0,8.0,9.0,8.0,15.0,12.0,16.0,12.0,10.0,4.0,...,29.0,36.0,35.0,22.0,29.0,36.0,35.0,22.0,30.0,34.0
540,10.0,9.0,9.0,8.0,15.0,12.0,13.0,12.0,8.0,8.0,...,30.0,24.0,36.0,31.0,30.0,25.0,36.0,30.0,30.0,24.0


## Preprocessing the results

In [12]:
weightlist = [
    (1, 0, 0),
    (0, 1, 0),
    (0, 0, 1),
    (0.5, 0.5, 0),
    (0, 0.5, 0.5),
    (0.5, 0, 0.5),
    (0.33, 0.33, 0.33),
    (0, 0, 0)
]

demands = [
    450,
    900,
    (450, 900),
    (900, 1800)
]

_model_type = {
    (1, 0, 0): 'Delay priority',
    (0, 1, 0): 'Throughput priority',
    (0, 0, 1): 'Flow priority',
    (0.5, 0.5, 0): 'Delay-Throughput priority',
    (0, 0.5, 0.5): 'Throughput-Flow priority',
    (0.5, 0, 0.5): 'Delay-Flow priority',
    (0.33, 0.33, 0.33): 'Equal priority',
    (0, 0, 0): 'Parent model',
}

In [13]:
total_thru_dfs = []
total_delay_dfs = []
ave_delay_dfs = []

for demand in demands:
    for weights in weightlist:

        partial_fn = 'old' if weights == (0, 0, 0) else 'a{}_b{}_c{}'.format(*weights)
        filename = currdir + '/experiments/sim-results/result_d{}_{}.xls'.format(demand, partial_fn)
        df_tmp = pd.read_excel(filename, sheet_name='Speed').fillna(0)

        # Get the cumulative delays

        df_tmp1 = df_tmp[df_tmp.t.isin(delay_rows)]
        df_tmp1['t'].replace(delay_rename, inplace=True)
        df_tmp1 = df_tmp1.sort_values(by='t').set_index('t')[entrances].stack().reset_index()
        df_tmp1['level_1'] = df_tmp1['level_1'].map(entrances_rename)
        df_tmp1 = df_tmp1.groupby(['t', 'level_1']).sum().reset_index()

        df_cumdelay = df_tmp1.pivot(index='t', columns='level_1', values=0)
        df_cumdelay.to_pickle(currdir + '/experiments/cumu-delay/delay_d{}_a{}_b{}_c{}.pkl'.format(demand, *weights))
        
        # Get the cumulative throughputs

        df_tmp3 = df_tmp[df_tmp.t.isin(thru_rows)]
#         if weights == (0, 0, 0):
#             for col in ['0x11 - exit3_lane1', '0x15 - exit1_lane1', '0x17 - exit0_lane1', '0x13 - exit2_lane1']:
#                 if col not in df_tmp3.columns:
#                     df_tmp3[col] = 0
        df_tmp3['t'].replace(thru_rename, inplace=True)
        df_tmp3 = df_tmp3.sort_values(by='t').set_index('t')[exits].stack().reset_index()
        df_tmp3['level_1'] = df_tmp3['level_1'].map(exits_rename)
        df_tmp3 = df_tmp3.groupby(['t', 'level_1']).sum().reset_index()

        df_cumthru = df_tmp3.pivot(index='t', columns='level_1', values=0)
        df_cumthru.to_pickle(currdir + '/experiments/cumu-thru/thru_d{}_a{}_b{}_c{}.pkl'.format(demand, *weights))
        
        # Get volumes
        
        df_tmp = pd.read_excel(filename, sheet_name='Volume').fillna(0)
        df_tmp1 = df_tmp[df_tmp.t.isin(vol_rows)]
        df_tmp1 = df_tmp1.sort_values(by='t').set_index('t')[entrances].stack().reset_index()
        df_tmp1['level_1'] = df_tmp1['level_1'].map(entrances_rename)
        df_tmp1 = df_tmp1.groupby(['t', 'level_1']).sum().reset_index()

        df_volume = df_tmp1.pivot(index='t', columns='level_1', values=0)
        df_volume.to_pickle(currdir + '/experiments/actual-volume/vol_d{}_a{}_b{}_c{}.pkl'.format(demand, *weights))

        # Get aggregate values

        df_total_thru_tmp = df_cumthru.loc[600, :].reset_index().set_index('level_1').T
        df_total_thru_tmp['demand'] = [demand]
        df_total_thru_tmp['new_model'] = weights != (0, 0, 0)
        df_total_thru_tmp['alpha'] = weights[0]
        df_total_thru_tmp['beta'] = weights[1]
        df_total_thru_tmp['gamma'] = weights[2]
        df_total_thru_tmp['model_type'] = _model_type[weights]
        total_thru_dfs.append(df_total_thru_tmp)

        df_total_delay_tmp = df_cumdelay.loc[600, :].reset_index().set_index('level_1').T
        df_total_delay_tmp['demand'] = [demand]
        df_total_delay_tmp['new_model'] = weights != (0, 0, 0)
        df_total_delay_tmp['alpha'] = weights[0]
        df_total_delay_tmp['beta'] = weights[1]
        df_total_delay_tmp['gamma'] = weights[2]
        df_total_delay_tmp['model_type'] = _model_type[weights]
        total_delay_dfs.append(df_total_delay_tmp)

        df_tmp2 = df_tmp[df_tmp.t == 'Average Delay'][entrances].rename(columns=entrances_rename).stack().reset_index()
        df_ave_delay_tmp = df_tmp2.groupby('level_1').mean().reset_index().pivot(index='level_0', columns='level_1', values=0)
        df_ave_delay_tmp['demand'] = [demand]
        df_ave_delay_tmp['new_model'] = weights != (0, 0, 0)
        df_ave_delay_tmp['alpha'] = weights[0]
        df_ave_delay_tmp['beta'] = weights[1]
        df_ave_delay_tmp['gamma'] = weights[2]
        df_ave_delay_tmp['model_type'] = _model_type[weights]
        ave_delay_dfs.append(df_ave_delay_tmp)

/home/localsim/.virtualenvs/thesis/local/lib/python2.7/site-packages/pandas/core/generic.py:6586: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [14]:
df_total_thru = pd.concat(total_thru_dfs).reset_index().iloc[:, 1:]
df_total_thru.to_pickle(currdir + '/experiments/total_throughput.pkl')

df_total_delay = pd.concat(total_delay_dfs).reset_index().iloc[:, 1:]
df_total_delay.to_pickle(currdir + '/experiments/total_delay.pkl')

df_ave_delay = pd.concat(ave_delay_dfs).reset_index().iloc[:, 1:]
df_ave_delay.to_pickle(currdir + '/experiments/ave_delay.pkl')